In [32]:
import os
import glob

import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_from_disk
from transformers import pipeline
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.preprocessing import MultiLabelBinarizer

LABELS = ('fairness_and_equality', 'security_and_defense', 'crime_and_punishment', 'morality',
          'policy_prescription_and_evaluation', 'capacity_and_resources', 'economic', 'cultural_identity',
          'health_and_safety', 'quality_of_life', 'legality_constitutionality_and_jurisprudence',
          'political', 'public_opinion', 'external_regulation_and_reputation')

# Join Datasets from different languages

In [10]:
df_paths = glob.glob(os.path.join('..', '..', 'data', 'preprocessed', '*train_and_dev.csv'))

In [11]:
for i, df_path_i in enumerate(df_paths):
    df_i = pd.read_csv(df_path_i, index_col='id')
    df_i['language'] = os.path.basename(df_path_i).split('_')[1]

    if i == 0:
        df = df_i
    else:
        df = pd.concat([df, df_i])

In [12]:
df

,frames,raw_text,title,content,title_and_5_sentences,title_and_10_sentences,title_and_first_paragraph,title_and_first_sentence_each_paragraph,language
id,,,,,,,,,
24151,"Legality_Constitutionality_and_jurisprudence,H...",Байдену напомнили о его отношению к абортам в ...,Байдену напомнили о его отношению к абортам в ...,CNN напомнил о трансформации отношения Байдена...,Байдену напомнили о его отношению к абортам в ...,Байдену напомнили о его отношению к абортам в ...,Байдену напомнили о его отношению к абортам в ...,CNN напомнил о трансформации отношения Байдена...,ru
24150,"Legality_Constitutionality_and_jurisprudence,H...",В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,В США после отмены права на аборт произошел на...,ru
24153,"Legality_Constitutionality_and_jurisprudence,F...","Байден пообещал американским мужчинам, что они...","Байден пообещал американским мужчинам, что они...",Решение Верховного суда об абортах ужасно. Не ...,"Байден пообещал американским мужчинам, что они...","Байден пообещал американским мужчинам, что они...","Байден пообещал американским мужчинам, что они...",Решение Верховного суда об абортах ужасно. «Эт...,ru
24152,"Legality_Constitutionality_and_jurisprudence,P...",Моё тело – моё дело: американки протестуют из-...,Моё тело – моё дело: американки протестуют из-...,Что случилось?\n\nВерховный суд США отменил ко...,Моё тело – моё дело: американки протестуют из-...,Моё тело – моё дело: американки протестуют из-...,Моё тело – моё дело: американки протестуют из-...,Что случилось? Верховный суд США отменил конст...,ru
24147,"Legality_Constitutionality_and_jurisprudence,E...",В США начались погромы из-за запрета абортов: ...,В США начались погромы из-за запрета абортов: ...,Почему скандальный закон приняли именно сейчас...,В США начались погромы из-за запрета абортов: ...,В США начались погромы из-за запрета абортов: ...,В США начались погромы из-за запрета абортов: ...,Почему скандальный закон приняли именно сейчас...,ru
...,...,...,...,...,...,...,...,...,...
25143,"Political,Security_and_defense","Kłopoty z dotarciem, czyli Objazdowy Cyrk Pana...","Kłopoty z dotarciem, czyli Objazdowy Cyrk Pana...","Każdy, kto miał kiedyś nowy samochód wie, że p...","Kłopoty z dotarciem, czyli Objazdowy Cyrk Pana...","Kłopoty z dotarciem, czyli Objazdowy Cyrk Pana...","Kłopoty z dotarciem, czyli Objazdowy Cyrk Pana...","Każdy, kto miał kiedyś nowy samochód wie, że p...",po
2528,"Policy_prescription_and_evaluation,Economic,Ex...",Rosja może uniknąć bankructwa. Zachodnia machi...,Rosja może uniknąć bankructwa. Zachodnia machi...,"Nie jest pewne, czy obecne sankcje wystarczą d...",Rosja może uniknąć bankructwa. Zachodnia machi...,Rosja może uniknąć bankructwa. Zachodnia machi...,Rosja może uniknąć bankructwa. Zachodnia machi...,"Nie jest pewne, czy obecne sankcje wystarczą d...",po
2530,"Policy_prescription_and_evaluation,Economic,Ex...",Embargo na gaz i ropę z Rosji? Jednoznaczne st...,Embargo na gaz i ropę z Rosji? Jednoznaczne st...,"Marine Le Pen, rywalka Emmanuela Macrona w wyś...",Embargo na gaz i ropę z Rosji? Jednoznaczne st...,Embargo na gaz i ropę z Rosji? Jednoznaczne st...,Embargo na gaz i ropę z Rosji? Jednoznaczne st...,"Marine Le Pen, rywalka Emmanuela Macrona w wyś...",po


### Encode labels

In [24]:
mlb = MultiLabelBinarizer()
mlb.fit([LABELS])

labels_npy = mlb.transform(df.frames.str.lower().str.split(','))
df['label'] = [list(labels_npy[i, :]) for i in range(labels_npy.shape[0])]

### Do iterative stratification to create a train and holdout set, stratifying per language

In [58]:
splits = 3
mskf = MultilabelStratifiedKFold(n_splits=splits, shuffle=True, random_state=0)

train_dfs = []
test_dfs = []
for language, df_ in df.groupby('language'):
    X = df_[[col for col in df.columns if col not in ['label', 'frames']]]
    y = df_[[col for col in df.columns if col in ['label', 'frames']]]

    for train_index, test_index in mskf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        train_dfs.append(X_train.join(y_train))
        test_dfs.append(X_test.join(y_test))

        break

train_df = pd.concat(train_dfs)
test_df = pd.concat(test_dfs)

### Create Dataset object

In [68]:
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

dataset = DatasetDict({'train': train_ds, 'test': test_ds})

In [69]:
dataset

DatasetDict({
    train: Dataset({
        features: ['raw_text', 'title', 'content', 'title_and_5_sentences', 'title_and_10_sentences', 'title_and_first_paragraph', 'title_and_first_sentence_each_paragraph', 'language', 'frames', 'label', 'id'],
        num_rows: 1059
    })
    test: Dataset({
        features: ['raw_text', 'title', 'content', 'title_and_5_sentences', 'title_and_10_sentences', 'title_and_first_paragraph', 'title_and_first_sentence_each_paragraph', 'language', 'frames', 'label', 'id'],
        num_rows: 530
    })
})

#### Save it

In [74]:
dataset.save_to_disk(os.path.join('..', '..', 'data', 'preprocessed','multilingual_train_test_ds.hf'))

Saving the dataset (0/1 shards):   0%|          | 0/1059 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/530 [00:00<?, ? examples/s]

# Test a model

In [86]:
id2label= {idx:label for idx, label in enumerate(mlb.classes_)}
label2id = {label:idx for idx, label in enumerate(mlb.classes_)}

In [81]:
dataset = load_from_disk(os.path.join('..', '..', 'data', 'preprocessed','multilingual_train_test_ds.hf'))

In [10]:
classifier = pipeline(model='xlm-roberta-large', task='zero-shot-classification')

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ImportError: 
XLMRobertaConverter requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [9]:
classifier = pipeline(model="joeddav/xlm-roberta-large-xnli", task="zero-shot-classification")

NameError: name 'transformers' is not defined

In [29]:
classifier = transformers.pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
libssl.so.10: cannot open shared object file: No such file or directory